In [1]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz

In [4]:
!tar xf spark-3.1.1-bin-hadoop2.7.tgz

In [5]:
!pip install -q findspark

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [7]:
import findspark
findspark.init()

In [8]:

!wget -O veh -q https://data.cityofnewyork.us/api/views/bm4k-52h4/rows.csv?accessType=DOWNLOAD


In [9]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -y ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2021-05-07 16:06:19--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.209.238.187, 35.169.70.242, 54.80.88.238, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.209.238.187|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  23.8MB/s    in 0.6s    

2021-05-07 16:06:19 (23.8 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

UnZip 6.00 of 20 April 2009, by Debian. Original by Info-ZIP.

Usage: unzip [-Z] [-opts[modifiers]] file[.zip] [list] [-x xlist] [-d exdir]
  Default action is to extract files in list, except those in xlist, to exdir;
  file[.zip] may be a wildcard.  -Z => ZipInfo mode ("unzip -Z" for usage).

  -p  extract files to pipe, no messages     -l  list files (short format)
  -f  freshen existing files, create none    -t  te

In [10]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Collision")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [11]:
col=spark.read.option("header",True).csv("./veh",inferSchema=True);

In [12]:
col.schema

StructType(List(StructField(UNIQUE_ID,IntegerType,true),StructField(COLLISION_ID,IntegerType,true),StructField(CRASH_DATE,StringType,true),StructField(CRASH_TIME,StringType,true),StructField(VEHICLE_ID,StringType,true),StructField(STATE_REGISTRATION,StringType,true),StructField(VEHICLE_TYPE,StringType,true),StructField(VEHICLE_MAKE,StringType,true),StructField(VEHICLE_MODEL,StringType,true),StructField(VEHICLE_YEAR,IntegerType,true),StructField(TRAVEL_DIRECTION,StringType,true),StructField(VEHICLE_OCCUPANTS,IntegerType,true),StructField(DRIVER_SEX,StringType,true),StructField(DRIVER_LICENSE_STATUS,StringType,true),StructField(DRIVER_LICENSE_JURISDICTION,StringType,true),StructField(PRE_CRASH,StringType,true),StructField(POINT_OF_IMPACT,StringType,true),StructField(VEHICLE_DAMAGE,StringType,true),StructField(VEHICLE_DAMAGE_1,StringType,true),StructField(VEHICLE_DAMAGE_2,StringType,true),StructField(VEHICLE_DAMAGE_3,StringType,true),StructField(PUBLIC_PROPERTY_DAMAGE,StringType,true),Str

In [13]:
col.printSchema()

root
 |-- UNIQUE_ID: integer (nullable = true)
 |-- COLLISION_ID: integer (nullable = true)
 |-- CRASH_DATE: string (nullable = true)
 |-- CRASH_TIME: string (nullable = true)
 |-- VEHICLE_ID: string (nullable = true)
 |-- STATE_REGISTRATION: string (nullable = true)
 |-- VEHICLE_TYPE: string (nullable = true)
 |-- VEHICLE_MAKE: string (nullable = true)
 |-- VEHICLE_MODEL: string (nullable = true)
 |-- VEHICLE_YEAR: integer (nullable = true)
 |-- TRAVEL_DIRECTION: string (nullable = true)
 |-- VEHICLE_OCCUPANTS: integer (nullable = true)
 |-- DRIVER_SEX: string (nullable = true)
 |-- DRIVER_LICENSE_STATUS: string (nullable = true)
 |-- DRIVER_LICENSE_JURISDICTION: string (nullable = true)
 |-- PRE_CRASH: string (nullable = true)
 |-- POINT_OF_IMPACT: string (nullable = true)
 |-- VEHICLE_DAMAGE: string (nullable = true)
 |-- VEHICLE_DAMAGE_1: string (nullable = true)
 |-- VEHICLE_DAMAGE_2: string (nullable = true)
 |-- VEHICLE_DAMAGE_3: string (nullable = true)
 |-- PUBLIC_PROPERTY_DAM

In [14]:
col.select("CRASH_TIME").show(10)


+----------+
|CRASH_TIME|
+----------+
|      9:03|
|      8:15|
|     17:18|
|     20:34|
|     21:15|
|     17:35|
|     13:15|
|     12:39|
|     11:20|
|     18:12|
+----------+
only showing top 10 rows



In [15]:
## Converting CRASH TIME type: String to timestamp 
from pyspark.sql.types import DateType
from pyspark.sql.functions import to_timestamp

col= col.withColumn('CRASH_TIME',to_timestamp(col["CRASH_TIME"], 'H:mm'))


In [16]:
col.select("CRASH_TIME").show(10)

+-------------------+
|         CRASH_TIME|
+-------------------+
|1970-01-01 09:03:00|
|1970-01-01 08:15:00|
|1970-01-01 17:18:00|
|1970-01-01 20:34:00|
|1970-01-01 21:15:00|
|1970-01-01 17:35:00|
|1970-01-01 13:15:00|
|1970-01-01 12:39:00|
|1970-01-01 11:20:00|
|1970-01-01 18:12:00|
+-------------------+
only showing top 10 rows



In [17]:

### Removing YYYY-MM-DD from CRASH TIME
from pyspark.sql.types import DateType
from pyspark.sql.functions import date_format
from pyspark.sql import functions as f

col = col.withColumn("CRASH_TIME", f.date_format("CRASH_TIME", 'HH:mm:ss'))

In [18]:
#Change CRASH DATE type from String to Date
from pyspark.sql.types import DateType
from pyspark.sql.functions import to_date

#col.withColumn("CRASH DATE", col['CRASH DATE'].cast(DateType())).show()

 
col= col.withColumn('CRASH_DATE',to_date(col["CRASH_DATE"], 'MM/dd/yyyy'))
col.select("CRASH_DATE").show(10)

+----------+
|CRASH_DATE|
+----------+
|2012-09-07|
|2019-09-23|
|2015-10-02|
|2015-10-04|
|2013-04-25|
|2016-05-02|
|2019-10-24|
|2016-08-18|
|2013-07-16|
|2012-11-26|
+----------+
only showing top 10 rows



In [19]:
col.printSchema()

root
 |-- UNIQUE_ID: integer (nullable = true)
 |-- COLLISION_ID: integer (nullable = true)
 |-- CRASH_DATE: date (nullable = true)
 |-- CRASH_TIME: string (nullable = true)
 |-- VEHICLE_ID: string (nullable = true)
 |-- STATE_REGISTRATION: string (nullable = true)
 |-- VEHICLE_TYPE: string (nullable = true)
 |-- VEHICLE_MAKE: string (nullable = true)
 |-- VEHICLE_MODEL: string (nullable = true)
 |-- VEHICLE_YEAR: integer (nullable = true)
 |-- TRAVEL_DIRECTION: string (nullable = true)
 |-- VEHICLE_OCCUPANTS: integer (nullable = true)
 |-- DRIVER_SEX: string (nullable = true)
 |-- DRIVER_LICENSE_STATUS: string (nullable = true)
 |-- DRIVER_LICENSE_JURISDICTION: string (nullable = true)
 |-- PRE_CRASH: string (nullable = true)
 |-- POINT_OF_IMPACT: string (nullable = true)
 |-- VEHICLE_DAMAGE: string (nullable = true)
 |-- VEHICLE_DAMAGE_1: string (nullable = true)
 |-- VEHICLE_DAMAGE_2: string (nullable = true)
 |-- VEHICLE_DAMAGE_3: string (nullable = true)
 |-- PUBLIC_PROPERTY_DAMAG

In [20]:
col.count()


3559738

In [21]:
col.filter(col.COLLISION_ID.isNull()).drop()

DataFrame[UNIQUE_ID: int, COLLISION_ID: int, CRASH_DATE: date, CRASH_TIME: string, VEHICLE_ID: string, STATE_REGISTRATION: string, VEHICLE_TYPE: string, VEHICLE_MAKE: string, VEHICLE_MODEL: string, VEHICLE_YEAR: int, TRAVEL_DIRECTION: string, VEHICLE_OCCUPANTS: int, DRIVER_SEX: string, DRIVER_LICENSE_STATUS: string, DRIVER_LICENSE_JURISDICTION: string, PRE_CRASH: string, POINT_OF_IMPACT: string, VEHICLE_DAMAGE: string, VEHICLE_DAMAGE_1: string, VEHICLE_DAMAGE_2: string, VEHICLE_DAMAGE_3: string, PUBLIC_PROPERTY_DAMAGE: string, PUBLIC_PROPERTY_DAMAGE_TYPE: string, CONTRIBUTING_FACTOR_1: string, CONTRIBUTING_FACTOR_2: string]

In [22]:
col.count()

3559738

In [23]:
missingId=col.filter(col["VEHICLE_ID"].isNull()).filter(col["UNIQUE_ID"].isNull())

In [24]:
cannotRetriveId = missingId.filter(col["COLLISION_ID"].isNull())


In [25]:
col.createOrReplaceTempView("col");
missingId.createOrReplaceTempView("toDelete")

In [26]:
col=spark.sql("select * FROM col WHERE col.COLLISION_ID not in (select COLLISION_ID from toDelete where COLLISION_ID is not null)");


In [27]:
col.filter(col.VEHICLE_ID.isNull()).drop()

DataFrame[UNIQUE_ID: int, COLLISION_ID: int, CRASH_DATE: date, CRASH_TIME: string, VEHICLE_ID: string, STATE_REGISTRATION: string, VEHICLE_TYPE: string, VEHICLE_MAKE: string, VEHICLE_MODEL: string, VEHICLE_YEAR: int, TRAVEL_DIRECTION: string, VEHICLE_OCCUPANTS: int, DRIVER_SEX: string, DRIVER_LICENSE_STATUS: string, DRIVER_LICENSE_JURISDICTION: string, PRE_CRASH: string, POINT_OF_IMPACT: string, VEHICLE_DAMAGE: string, VEHICLE_DAMAGE_1: string, VEHICLE_DAMAGE_2: string, VEHICLE_DAMAGE_3: string, PUBLIC_PROPERTY_DAMAGE: string, PUBLIC_PROPERTY_DAMAGE_TYPE: string, CONTRIBUTING_FACTOR_1: string, CONTRIBUTING_FACTOR_2: string]

In [28]:
print(col.select("VEHICLE_ID").distinct().show(100));

+--------------------+
|          VEHICLE_ID|
+--------------------+
|              692423|
|              370609|
|f015cba0-adcb-48c...|
|a4248672-101e-406...|
|c06970b5-afa0-4ea...|
|023ab94e-76e4-4ca...|
|57b5ffa1-20b5-4df...|
|2d8a816f-d14a-44d...|
|              505944|
|              147348|
|              555779|
|              113602|
|             1108583|
|             1219138|
|             1218061|
|              642225|
|              762357|
|             1172127|
|             1155583|
|              502936|
|f1548baf-d7cc-4da...|
|              621533|
|              622985|
|              371086|
|              762037|
|              615275|
|             1214729|
|              556677|
|              808131|
|              768779|
|              124057|
|               29549|
|              321607|
|              892775|
|             1086716|
|              427979|
|              830532|
|              690002|
|             1176173|
|              722232|
|          

In [29]:
print(col.select("VEHICLE_DAMAGE_1","VEHICLE_DAMAGE_2","VEHICLE_DAMAGE_3").distinct().show(30));
## Checking if null value is present

+--------------------+--------------------+--------------------+
|    VEHICLE_DAMAGE_1|    VEHICLE_DAMAGE_2|    VEHICLE_DAMAGE_3|
+--------------------+--------------------+--------------------+
|    Right Side Doors|Right Rear Quarte...|               Other|
|Left Front Quarte...|Left Rear Quarter...|           No Damage|
|     Left Side Doors|   Left Front Bumper|Right Front Quart...|
|   Left Front Bumper|  Right Front Bumper|Right Rear Quarte...|
|   Left Front Bumper|    Center Front End|       Undercarriage|
|   Left Front Bumper|    Center Front End|     Left Side Doors|
|Right Rear Quarte...|       Undercarriage|           No Damage|
|     Center Back End|   Right Rear Bumper|     Left Side Doors|
|    Right Side Doors|   Left Front Bumper|    Left Rear Bumper|
|     Center Back End|   Left Front Bumper|                Roof|
|   Left Front Bumper|Right Rear Quarte...|           No Damage|
|  Right Front Bumper|Left Front Quarte...|                Roof|
|           No Damage|   

In [30]:
col.na.drop(subset=["DRIVER_LICENSE_STATUS"]) \
   .show(truncate=False)

+---------+------------+----------+----------+------------------------------------+------------------+-----------------------------------+--------------+-------------+------------+----------------+-----------------+----------+---------------------+---------------------------+------------------------+------------------------+------------------------+-------------------------+-------------------------+------------------+----------------------+---------------------------+------------------------------+------------------------------+
|UNIQUE_ID|COLLISION_ID|CRASH_DATE|CRASH_TIME|VEHICLE_ID                          |STATE_REGISTRATION|VEHICLE_TYPE                       |VEHICLE_MAKE  |VEHICLE_MODEL|VEHICLE_YEAR|TRAVEL_DIRECTION|VEHICLE_OCCUPANTS|DRIVER_SEX|DRIVER_LICENSE_STATUS|DRIVER_LICENSE_JURISDICTION|PRE_CRASH               |POINT_OF_IMPACT         |VEHICLE_DAMAGE          |VEHICLE_DAMAGE_1         |VEHICLE_DAMAGE_2         |VEHICLE_DAMAGE_3  |PUBLIC_PROPERTY_DAMAGE|PUBLIC_PROPERTY_DAM

In [31]:
print(col.select("DRIVER_LICENSE_STATUS").distinct().show(30));


+---------------------+
|DRIVER_LICENSE_STATUS|
+---------------------+
|                 null|
|           Unlicensed|
|               Permit|
|             Licensed|
+---------------------+

None


In [32]:

## Setting all null values to Unknown 
col.filter(col.COLLISION_ID.isNull()).drop()

DataFrame[UNIQUE_ID: int, COLLISION_ID: int, CRASH_DATE: date, CRASH_TIME: string, VEHICLE_ID: string, STATE_REGISTRATION: string, VEHICLE_TYPE: string, VEHICLE_MAKE: string, VEHICLE_MODEL: string, VEHICLE_YEAR: int, TRAVEL_DIRECTION: string, VEHICLE_OCCUPANTS: int, DRIVER_SEX: string, DRIVER_LICENSE_STATUS: string, DRIVER_LICENSE_JURISDICTION: string, PRE_CRASH: string, POINT_OF_IMPACT: string, VEHICLE_DAMAGE: string, VEHICLE_DAMAGE_1: string, VEHICLE_DAMAGE_2: string, VEHICLE_DAMAGE_3: string, PUBLIC_PROPERTY_DAMAGE: string, PUBLIC_PROPERTY_DAMAGE_TYPE: string, CONTRIBUTING_FACTOR_1: string, CONTRIBUTING_FACTOR_2: string]

In [33]:
print(col.select("DRIVER_LICENSE_JURISDICTION").distinct().show(30));
## Checking if null value is present

+---------------------------+
|DRIVER_LICENSE_JURISDICTION|
+---------------------------+
|                   A,NEJADE|
|                         SC|
|                         AZ|
|                        ZZZ|
|                         NS|
|                         LA|
|                         MN|
|                         NJ|
|                         MX|
|                         DC|
|                         OR|
|                         NT|
|                       null|
|                         VA|
|                        NB1|
|                         RI|
|                         KY|
|                         WY|
|                         BC|
|                         NH|
|                         MI|
|                         NV|
|                         WI|
|                         ID|
|                         CA|
|                         CT|
|                         NE|
|                         MT|
|                         NC|
|                         PQ|
+---------

In [34]:
## Setting all null values to Unknown 
col=col.fillna(value="Not Provided",subset=["DRIVER_LICENSE_JURISDICTION"])

In [35]:
print(col.select("POINT_OF_IMPACT").distinct().show(30));
## Checking if null value is present

+--------------------+
|     POINT_OF_IMPACT|
+--------------------+
|Left Rear Quarter...|
|     Center Back End|
|Left Front Quarte...|
|                null|
|  Right Front Bumper|
|    Left Rear Bumper|
|    Right Side Doors|
|                Roof|
|Right Front Quart...|
|               Other|
|          Overturned|
|             Trailer|
|Right Rear Quarte...|
|     Left Side Doors|
|   Left Front Bumper|
|          Demolished|
|    Center Front End|
|       Undercarriage|
|   Right Rear Bumper|
|           No Damage|
+--------------------+

None


In [36]:
## Setting all null values to Unknown 
col=col.fillna(value="Not Provided",subset=["POINT_OF_IMPACT"])

In [37]:
print(col.select("VEHICLE_TYPE","VEHICLE_MAKE","VEHICLE_MODEL").distinct().show(30));
## Checking if null value is present

+--------------------+----------------+-------------+
|        VEHICLE_TYPE|    VEHICLE_MAKE|VEHICLE_MODEL|
+--------------------+----------------+-------------+
|Tractor Truck Diesel|           DORSE|         null|
|Station Wagon/Spo...|   JEEP -CAR/SUV|     JEEP GCH|
|           Box Truck|  INTL-TRUCK/BUS|         4400|
|Station Wagon/Spo...|   JEEP -CAR/SUV|     JEEP PAT|
|               Sedan|           WAGON|         null|
|          4 dr sedan|   ACUR -CAR/SUV|          ATL|
|           Box Truck|           WORKH|         null|
|          Motorcycle|       HOND -MCL|          CTX|
|                null|         UNKNOWN|      UNKNOWN|
|Tractor Truck Gas...|  FRHT-TRUCK/BUS|      FRHT TK|
|               trail|            null|         null|
|               Sedan|   HUMM -CAR/SUV|         null|
|Station Wagon/Spo...|           HONDA|         null|
|                 Van|             unk|         null|
|       Pick-up Truck|             ihc|         null|
|                 Bus|      

In [38]:
## Setting all null values to Unknown 
col=col.fillna(value="Not Provided",subset=["VEHICLE_TYPE","VEHICLE_MAKE","VEHICLE_MODEL"])

In [39]:
print(col.select("STATE_REGISTRATION").distinct().show(100));
## Checking if null value is present

+------------------+
|STATE_REGISTRATION|
+------------------+
|                SC|
|                AZ|
|                UA|
|                NS|
|                LA|
|                MN|
|                NJ|
|                MX|
|                DC|
|                ZZ|
|                OR|
|                NT|
|              null|
|                VA|
|                RI|
|                WY|
|                KY|
|                BC|
|                NH|
|                LR|
|                MI|
|                NV|
|                GV|
|                QB|
|                WI|
|                ID|
|                CA|
|                CT|
|                NE|
|                MT|
|                NC|
|                VT|
|                PQ|
|                MD|
|                DE|
|                MO|
|                VI|
|                IL|
|                ME|
|                MB|
|                GU|
|                WA|
|                ND|
|                MS|
|            

In [40]:
print(col.select("TRAVEL_DIRECTION").distinct().show(20));
## Checking if null value is present

+----------------+
|TRAVEL_DIRECTION|
+----------------+
|            null|
|               E|
|       Northwest|
|               U|
|         Unknown|
|           South|
|       Southeast|
|               -|
|               N|
|               W|
|               S|
|            East|
|       Southwest|
|            West|
|           North|
|       Northeast|
+----------------+

None


In [41]:
## Setting all null values to Unknown 
col=col.fillna(value="unknown",subset=["TRAVEL_DIRECTION"])

In [42]:
print(col.select("VEHICLE_OCCUPANTS").distinct().show(20));

+-----------------+
|VEHICLE_OCCUPANTS|
+-----------------+
|              148|
|              392|
|               31|
|          9999999|
|               65|
|               53|
|               34|
|              211|
|               81|
|               28|
|              183|
|        999999999|
|               76|
|               26|
|               27|
|               44|
|              743|
|               12|
|               22|
|             2013|
+-----------------+
only showing top 20 rows

None


In [43]:
print(col.select("DRIVER_SEX").distinct().show(20));

+----------+
|DRIVER_SEX|
+----------+
|         F|
|      null|
|         M|
|         U|
+----------+

None


In [44]:
col=col.fillna(value="Not Provided",subset=["DRIVER_SEX"])

In [45]:
print(col.select("DRIVER_LICENSE_STATUS").distinct().show(20));

+---------------------+
|DRIVER_LICENSE_STATUS|
+---------------------+
|                 null|
|           Unlicensed|
|               Permit|
|             Licensed|
+---------------------+

None


In [46]:
print(col.select("DRIVER_LICENSE_JURISDICTION").distinct().show(20));

+---------------------------+
|DRIVER_LICENSE_JURISDICTION|
+---------------------------+
|                   A,NEJADE|
|                         SC|
|                         AZ|
|                        ZZZ|
|                         NS|
|                         LA|
|                         MN|
|                         NJ|
|                         MX|
|                         DC|
|                         OR|
|                         NT|
|                         VA|
|                        NB1|
|                         RI|
|                         KY|
|                         WY|
|                         BC|
|                         NH|
|                         MI|
+---------------------------+
only showing top 20 rows

None


In [47]:
print(col.select("PRE_CRASH").distinct().show(20));

+--------------------+
|           PRE_CRASH|
+--------------------+
|Making Right Turn...|
|             Merging|
|                null|
| Starting in Traffic|
|             Passing|
|       Making U Turn|
|Starting from Par...|
|  Stopped in Traffic|
|              Other*|
|             Backing|
|Making Left Turn ...|
|    Making Left Turn|
|Entering Parked P...|
|Going Straight Ahead|
|      Police Pursuit|
|      Changing Lanes|
| Slowing or Stopping|
|Avoiding Object i...|
|              Parked|
|   Making Right Turn|
+--------------------+

None


In [48]:
col=col.fillna(value="Not Provided",subset=["PRE_CRASH"])

In [49]:
print(col.select("POINT_OF_IMPACT").distinct().show(30));

+--------------------+
|     POINT_OF_IMPACT|
+--------------------+
|Left Rear Quarter...|
|     Center Back End|
|Left Front Quarte...|
|  Right Front Bumper|
|    Left Rear Bumper|
|    Right Side Doors|
|                Roof|
|Right Front Quart...|
|               Other|
|          Overturned|
|             Trailer|
|Right Rear Quarte...|
|     Left Side Doors|
|   Left Front Bumper|
|          Demolished|
|    Center Front End|
|       Undercarriage|
|   Right Rear Bumper|
|        Not Provided|
|           No Damage|
+--------------------+

None


In [50]:
print(col.select("PUBLIC_PROPERTY_DAMAGE","PUBLIC_PROPERTY_DAMAGE_TYPE").distinct().show(30));

+----------------------+---------------------------+
|PUBLIC_PROPERTY_DAMAGE|PUBLIC_PROPERTY_DAMAGE_TYPE|
+----------------------+---------------------------+
|           Unspecified|       FENCE OF 503 CARY...|
|           Unspecified|       A NO STANDING TRA...|
|           Unspecified|        UTILITY LIGHT POST.|
|           Unspecified|       DAMAGED TREE GUAR...|
|           Unspecified|       TRAFFIC CONTROL D...|
|           Unspecified|              NYPD VMS SIGN|
|           Unspecified|               HOUSE GARAGE|
|           Unspecified|       DENTS AND SCRATCH...|
|           Unspecified|       FENCE OF 853 EAST...|
|           Unspecified|       TRAFFIC SIGNAL DE...|
|           Unspecified|       BRICK WALL 2FT X 8FT|
|           Unspecified|       NOTIFIED DEPARTME...|
|           Unspecified|       METAL GATE IN FRO...|
|           Unspecified|       FRONT GATE TO ENT...|
|           Unspecified|       DAMAGE TO BUILDIN...|
|           Unspecified|       LIGHT POLE ON T

In [51]:
col=col.fillna(value="Not Provided",subset=["PUBLIC_PROPERTY_DAMAGE"])

In [52]:
col.printSchema()

root
 |-- UNIQUE_ID: integer (nullable = true)
 |-- COLLISION_ID: integer (nullable = true)
 |-- CRASH_DATE: date (nullable = true)
 |-- CRASH_TIME: string (nullable = true)
 |-- VEHICLE_ID: string (nullable = true)
 |-- STATE_REGISTRATION: string (nullable = true)
 |-- VEHICLE_TYPE: string (nullable = false)
 |-- VEHICLE_MAKE: string (nullable = false)
 |-- VEHICLE_MODEL: string (nullable = false)
 |-- VEHICLE_YEAR: integer (nullable = true)
 |-- TRAVEL_DIRECTION: string (nullable = false)
 |-- VEHICLE_OCCUPANTS: integer (nullable = true)
 |-- DRIVER_SEX: string (nullable = false)
 |-- DRIVER_LICENSE_STATUS: string (nullable = true)
 |-- DRIVER_LICENSE_JURISDICTION: string (nullable = false)
 |-- PRE_CRASH: string (nullable = false)
 |-- POINT_OF_IMPACT: string (nullable = false)
 |-- VEHICLE_DAMAGE: string (nullable = true)
 |-- VEHICLE_DAMAGE_1: string (nullable = true)
 |-- VEHICLE_DAMAGE_2: string (nullable = true)
 |-- VEHICLE_DAMAGE_3: string (nullable = true)
 |-- PUBLIC_PROPER

In [53]:
pip install openclean


     |████████████████████████████████| 276kB 6.2MB/s 
     |████████████████████████████████| 92kB 4.9MB/s 
     |████████████████████████████████| 235kB 9.4MB/s 
     |████████████████████████████████| 112kB 10.9MB/s 
     |████████████████████████████████| 61kB 5.5MB/s 
     |████████████████████████████████| 645kB 9.9MB/s 
     |████████████████████████████████| 163kB 17.7MB/s 
     |████████████████████████████████| 532kB 18.2MB/s 
     |████████████████████████████████| 153kB 23.8MB/s 
     |████████████████████████████████| 71kB 6.7MB/s 
  Created wheel for datasize: filename=datasize-1.0.0-py2.py3-none-any.whl size=155029 sha256=cb2908fe7b058057bd661f4de9ef8c5e5ca69d92b59af74c1b0ebccf3c0ec6e5
  Stored in directory: /root/.cache/pip/wheels/b9/a1/a3/97bc5bd191a248dc72fe842abb780935d79fdf9aeee265f1e8
Successfully built datasize
ERROR: nbclient 0.5.3 has requirement jupyter-client>=6.1.5, but you'll have jupyter-client 5.3.5 which is incompatible.
  Found existing installation: PyY

In [54]:
import gzip
import os

datafile = './veh'

In [55]:
from openclean.pipeline import stream

ds = stream(datafile)

In [56]:
ds.head()


,UNIQUE_ID,COLLISION_ID,CRASH_DATE,CRASH_TIME,VEHICLE_ID,STATE_REGISTRATION,VEHICLE_TYPE,VEHICLE_MAKE,VEHICLE_MODEL,VEHICLE_YEAR,TRAVEL_DIRECTION,VEHICLE_OCCUPANTS,DRIVER_SEX,DRIVER_LICENSE_STATUS,DRIVER_LICENSE_JURISDICTION,PRE_CRASH,POINT_OF_IMPACT,VEHICLE_DAMAGE,VEHICLE_DAMAGE_1,VEHICLE_DAMAGE_2,VEHICLE_DAMAGE_3,PUBLIC_PROPERTY_DAMAGE,PUBLIC_PROPERTY_DAMAGE_TYPE,CONTRIBUTING_FACTOR_1,CONTRIBUTING_FACTOR_2
0,10385780,100201,09/07/2012,9:03,1,NY,PASSENGER VEHICLE,,,,,,,,,,,,,,,,,Unspecified,
1,19140702,4213082,09/23/2019,8:15,0553ab4d-9500-4cba-8d98-f4d7f89d5856,NY,Station Wagon/Sport Utility Vehicle,TOYT -CAR/SUV,,2002,North,1,M,Licensed,NY,Going Straight Ahead,Left Front Bumper,Left Front Quarter Panel,,,,N,,Driver Inattention/Distraction,Unspecified
2,14887647,3307608,10/02/2015,17:18,2,NY,TAXI,,,,,,,,,Going Straight Ahead,,,,,,,,Driver Inattention/Distraction,
3,14889754,3308693,10/04/2015,20:34,1,NY,PASSENGER VEHICLE,,,,,,,,,Parked,,,,,,,,Unspecified,
4,14400270,297666,04/25/2013,21:15,1,NY,PASSENGER VEHICLE,,,,,,,,,,,,,,,,,Other Vehicular,
5,17044639,3434155,05/02/2016,17:35,219456,NY,4 dr sedan,MERZ -CAR/SUV,,2015,East,2,M,Licensed,FL,Merging,Right Front Bumper,Right Front Bumper,Right Front Quarter Panel,,,N,,Driver Inattention/Distraction,Unsafe Lane Changing
6,19138701,4229067,10/24/2019,13:15,c53b43d9-419a-4ab1-9361-3f2979078d89,NY,Bus,FRHT-TRUCK/BUS,,2006,East,13,M,Licensed,NY,Parked,Left Front Quarter Panel,Left Front Quarter Panel,,,,N,,Unspecified,Unspecified
7,17303317,3503027,08/18/2016,12:39,672828,NY,Station Wagon/Sport Utility Vehicle,FORD -CAR/SUV,,2005,Southwest,2,F,Licensed,NY,Going Straight Ahead,Center Front End,Center Front End,No Damage,No Damage,No Damage,N,,Driver Inattention/Distraction,Unspecified
8,12254536,196425,07/16/2013,11:20,1,NY,PASSENGER VEHICLE,,,,,,,,,,,,,,,,,Unspecified,
9,11804847,2975897,11/26/2012,18:12,2,NY,PASSENGER VEHICLE,,,,,,,,,,,,,,,,,Driver Inattention/Distraction,


In [57]:
from openclean.profiling.column import DefaultColumnProfiler

profiles = ds\
    .sample(n=1000, random_state=42)\
    .profile(default_profiler=DefaultColumnProfiler)

In [58]:
# Print overview of profiling results.

profiles.stats()

,total,empty,distinct,uniqueness,entropy
UNIQUE_ID,1000,0,1000,1.000000,9.965784
COLLISION_ID,1000,0,998,0.998000,9.961784
CRASH_DATE,1000,0,831,0.831000,9.608687
CRASH_TIME,1000,0,416,0.416000,8.175374
VEHICLE_ID,1000,0,599,0.599000,6.934978
STATE_REGISTRATION,1000,62,23,0.024520,1.147366
VEHICLE_TYPE,1000,45,46,0.048168,3.389546
VEHICLE_MAKE,1000,471,67,0.126654,4.566605
VEHICLE_MODEL,1000,990,10,1.000000,3.321928
VEHICLE_YEAR,1000,479,27,0.051823,4.287315


In [59]:

# Get set of distinct values for column 'STATE_REGISTRATION'. Print the
# values in decreasing order of frequency.

states = ds.distinct('STATE_REGISTRATION')
for rank, val in enumerate(states.most_common()):
    st, freq = val
    print('{:<3} {}  {:>10}'.format('{}.'.format(rank + 1), st, '{:,}'.format(freq)))

1.  NY   2,832,197
2.  NJ     201,630
3.       190,129
4.  PA      70,687
5.  FL      38,270
6.  CT      36,650
7.  VA      16,227
8.  MA      15,745
9.  MD      15,574
10. NC      14,044
11. IN      11,004
12. GA      10,635
13. TX      10,001
14. IL       9,131
15. UK       8,888
16. AZ       8,298
17. SC       6,686
18. ZZ       6,631
19. OK       6,128
20. OH       5,432
21. DE       4,501
22. CA       4,054
23. TN       3,983
24. MI       2,995
25. ME       2,868
26. RI       2,792
27. NH       1,871
28. AL       1,748
29. VT       1,552
30. WA       1,406
31. MN       1,363
32. ON       1,298
33. WI       1,296
34. MO       1,217
35. NM       1,131
36. DC         907
37. IA         823
38. KY         816
39. LA         771
40. CO         768
41. MS         724
42. PQ         655
43. NB         625
44. WV         554
45. KS         540
46. AR         494
47. OR         470
48. NV         364
49. UT         343
50. SD         337
51. QB         333
52. MT         272
53. NF        

In [60]:
# List identifier and names for available reference datasets.

from openclean.data.refdata import RefStore

refdata = RefStore()
for entry in refdata.repository().find():
    print('{:<35}:  {}'.format(entry.identifier, entry.name))

company_suffixes                   :  Company Suffixes
encyclopaedia_britannica:us_cities :  Cities in the U.S.
nyc.gov:dof:state_codes            :  NYC Finance - State Codes
restcountries.eu                   :  REST Countries
usps:street_abbrev                 :  C1 Street Suffix Abbreviations
usps:secondary_unit_designators    :  C2 Secondary Unit Designators
wikipedia:us_states                :  States and territories of the U.S.


In [61]:
# Download the license plate state codes dataset.

refdata\
    .load('nyc.gov:dof:state_codes', auto_download=True)\
    .df()\
    .head()

,code,name,type
0,AL,Alabama,US
1,AK,Alaska,US
2,AZ,Arizona,US
3,AR,Arkansas,US
4,CA,California,US


In [62]:
# Get set of distinct state codes.

states_ref = refdata.load('nyc.gov:dof:state_codes', auto_download=True).distinct('code')

In [63]:

# Print information for entries in the 'STATE_REGISTRATION' column that
# do not occur in the reference dataset.

for rank, val in enumerate(states.most_common()):
    st, freq = val
    if st not in states_ref:
        print('{:<3} {}  {:>10}'.format('{}.'.format(rank + 1), st, '{:,}'.format(freq)))

3.       190,129
15. UK       8,888
18. ZZ       6,631
42. PQ         655
65. OT          40
72. VI          15
74. DI           5
75. FM           5
76. MH           4
77. GU           2
78. PR           2
79. MP           1
80. AS           1
81. UA           1
82. LR           1


In [64]:
VEHICLEMAKE = ds.distinct('VEHICLE_MAKE')

for rank, val in enumerate(VEHICLEMAKE.most_common(10)):
    st, freq = val
    print('{:<3} {:>8}  {:>10}'.format('{}.'.format(rank + 1), st, '{:,}'.format(freq)))

print('\nTotal number of distinct values is {}'.format(len(VEHICLEMAKE)))

1.             1,756,205
2.  TOYT -CAR/SUV     317,100
3.  HOND -CAR/SUV     219,231
4.  NISS -CAR/SUV     187,775
5.  FORD -CAR/SUV     154,186
6.  CHEV -CAR/SUV      88,387
7.  HYUN -CAR/SUV      63,562
8.  MERZ -CAR/SUV      57,896
9.  BMW -CAR/SUV      57,549
10. JEEP -CAR/SUV      56,903

Total number of distinct values is 9495


In [65]:
# Using the default settings yields two outliers.

from openclean.profiling.anomalies.sklearn import DBSCANOutliers

DBSCANOutliers().find(VEHICLEMAKE)

['', '.', '????']

In [66]:
# Cluster street names using 'Key Collision' clustering with the
# default fingerprint key generator.

from openclean.cluster.key import KeyCollision
from openclean.function.value.key.fingerprint import Fingerprint

Juridiction = ds.update('DRIVER_LICENSE_JURISDICTION', str.upper).distinct('DRIVER_LICENSE_JURISDICTION')
clusters = KeyCollision(func=Fingerprint()).clusters(Juridiction)

In [67]:

# Define simple helper method to print the k largest clusters.

def print_k_clusters(clusters, k=5):
    clusters = sorted(clusters, key=lambda x: len(x), reverse=True)
    val_count = sum([len(c) for c in clusters])
    print('Total number of clusters is {} with {} values'.format(len(clusters), val_count))
    for i in range(min(k, len(clusters))):
        print('\nCluster {}'.format(i + 1))
        for key, cnt in clusters[i].items():
            if key == '':
                key = "''"
            print('  {} (x {})'.format(key, cnt))

In [68]:
print_k_clusters(clusters)


Total number of clusters is 1 with 2 values

Cluster 1
  PA (x 26191)
  PA' (x 1)


In [69]:
# Cluster street names using 'Key Collision' clustering with the
# default fingerprint key generator.

from openclean.cluster.key import KeyCollision
from openclean.function.value.key.fingerprint import Fingerprint

vType = ds.update('VEHICLE_TYPE', str.upper).distinct('VEHICLE_TYPE')
clusterV = KeyCollision(func=Fingerprint()).clusters(vType)

In [70]:
# Define simple helper method to print the k largest clusters.

def print_k_clusters(clusterV, k=5):
    clusterV = sorted(clusterV, key=lambda x: len(x), reverse=True)
    val_count = sum([len(c) for c in clusterV])
    print('Total number of clusters is {} with {} values'.format(len(clusterV), val_count))
    for i in range(min(k, len(clusterV))):
        print('\nCluster {}'.format(i + 1))
        for key, cnt in clusterV[i].items():
            if key == '':
                key = "''"
            print('  {} (x {})'.format(key, cnt))

In [71]:
print_k_clusters(clusters)


Total number of clusters is 1 with 2 values

Cluster 1
  PA (x 26191)
  PA' (x 1)


In [72]:
print(col.select("UNIQUE_ID").distinct().show(30));

+---------+
|UNIQUE_ID|
+---------+
| 15041840|
| 19138693|
| 14751100|
| 19140680|
| 15078965|
| 19139771|
| 19139827|
| 19140022|
| 19139682|
| 13648716|
| 14912061|
| 12544877|
| 14731566|
|  9625628|
| 16935579|
| 17381522|
| 17385146|
| 11971827|
| 17173557|
| 14678441|
| 17413847|
| 11752058|
| 17148205|
| 17249982|
| 14932328|
| 17172550|
| 15018523|
| 17053776|
|  8478179|
| 16935603|
+---------+
only showing top 30 rows

None


In [73]:

from openclean.function.eval.base import Col
from openclean.function.eval.logic import And
from openclean.function.eval.null import IsNotEmpty

df = ds\
    .select(['UNIQUE_ID', 'STATE_REGISTRATION', 'VEHICLE_TYPE'])\
    .rename('STATE_REGISTRATION', 'State')\
    .where(And(IsNotEmpty('VEHICLE_TYPE'), Col('State') != '99'))\
    .update('VEHICLE_TYPE', str.upper)\
    .sample(n=100000, random_state=42)\
    .to_df()

In [74]:
from openclean.operator.map.violations import fd_violations


groups = fd_violations(df, lhs=['UNIQUE_ID', 'State'], rhs='VEHICLE_TYPE')


print('{} vehicles with conflicting Type'.format(len(groups)))

0 vehicles with conflicting Type


In [75]:

from openclean.function.eval.base import Col
from openclean.function.eval.logic import And
from openclean.function.eval.null import IsNotEmpty

df = ds\
    .select(['UNIQUE_ID', 'STATE_REGISTRATION', 'VEHICLE_MAKE'])\
    .rename('STATE_REGISTRATION', 'State')\
    .where(And(IsNotEmpty('VEHICLE_MAKE'), Col('State') != '99'))\
    .update('VEHICLE_MAKE', str.upper)\
    .sample(n=100000, random_state=42)\
    .to_df()

In [76]:
from openclean.operator.map.violations import fd_violations


groups2 = fd_violations(df, lhs=['UNIQUE_ID', 'State'], rhs='VEHICLE_MAKE')


print('{} vehicles with conflicting Make'.format(len(groups2)))

0 vehicles with conflicting Make


In [77]:
df

,UNIQUE_ID,State,VEHICLE_MAKE
2609371,17543896,NY,BUIC -CAR/SUV
1686978,17301972,NY,NISS -CAR/SUV
2392969,18726366,NY,KIA -CAR/SUV
3339147,19561739,NJ,CHEV -CAR/SUV
3340750,19645155,NY,JEEP -CAR/SUV
...,...,...,...
3333543,19651631,NY,CHRY -CAR/SUV
2239205,18608829,NY,TOYT -CAR/SUV
3297146,19802302,NY,DODG -CAR/SUV
1077156,17059184,MD,AUDI -CAR/SUV
